In [ ]:
from pyTigerGraph import TigerGraphConnection

# We first create a connection to the database
conn = TigerGraphConnection(
    host="https://YOUR_DB_ADDRESS",
    username="YOUR_DB_USERNAME", 
    password="YOUR_DB_PASSWORD")

# And then add CoPilot's address to the connection. This address 
# is the host's address where the CoPilot container is running.
conn.ai.configureCoPilotHost("http://COPILOT_ADDRESS")